<a href="https://colab.research.google.com/github/jimenaminino12/Mineria-datos/blob/main/Lab02_EJ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica	2	(Patrones	Secuenciales)**
**Objetivo	(5	puntos)**
El conjunto de datos “Online Retail.xlsx” contiene productos (definidos por la columna StockCode y
Description) comprados por diferentes clientes (definidos por la columna CustomerID) en diferentes
momentos (definidos por el campo InvoiceDate. Habrá que procesar el fichero para crear una lista de los
distintos clientes con las secuencias de las transacciones de las compras realizadas en distintos instantes (2
puntos). Una vez hecho esto y analizando esta información se pueden extraer conclusiones sobre en que orden
compran los clientes los productos y así tomar decisiones de negocio. Este estudio se puede llevar a cabo
aplicando el algoritmo Generalized Sequential Patterns utilizando la implementación de éste disponible en la
librería gsppy.

In [2]:
!pip install pandas
!pip install openpyxl
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install gsppy


In [3]:
from posixpath import sep
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from gsppy.gsp import GSP
import pandas as pd
from collections import defaultdict





## **1. Introducimos los datos y procedemos a su limpieza:**





Mediante el uso de la librería pandas implementamos los datos del archivo Online Retail.xlsx. El archivo contiene información sobre transacciones de ventas, acerca de  los productos comprados (identificados por el código de stock y la descripción), la fecha de la factura, el precio, el país y el identificador del cliente que realizó la compra.

In [4]:
data = pd.read_excel("Online Retail.xlsx")
# Limpiamos los datos
datos = data.dropna()  # Eliminamos filas con valores nulos

In [ ]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


Ordenamos las filas de datos de acuerdo a el identificador del cliente (CustomerID) y luego por la fecha de la factura (InvoiceDate). El resultado, que queda ordenado, lo guardamos en un nuevo DataFrame llamado datos_ordenados, es decir, lo ordenamos por secuencias.

In [5]:
# Ordenar las transacciones por CustomerID y luego por InvoiceDate
datos_ordenados = datos.sort_values(by=['CustomerID', 'InvoiceDate'])

## **2. Creación de transacciones**

Una vez ya tenemos nuestros datos ordenados y limpios crearemos las transacciones. Vamos a crear un diccionario llamado transaccion_cliente que contendrá las transacciones de cada cliente en forma de listas. Cada clave en el diccionario representará el CustomerID y su valor correspondiente será una lista de transacciones realizadas por ese cliente.

In [6]:
# Creamos una lista de transacciones para cada cliente
transaccion_cliente = {}

# Iteramos sobre cada grupo de transacciones por cliente
for customer_id, group in datos_ordenados.groupby('CustomerID'):

    # Obtenemos una lista de transacciones ordenadas por fecha para este cliente
    transacciones = group[['Description', 'InvoiceDate']].values.tolist()

    # Agregamos la lista de transaciones como una secuencia para este cliente en el diccionario
    transaccion_cliente[customer_id] = transacciones

Imprimimos un ejemplo de las transacciones realizadas por cada cliente del conjunto de datos. Esto será muy útil para reafirmarnos y asegurarnos que las transacciones son correctas. Al verificar las transacciones, podremos comprobar si están ordenadas cronológicamente.

In [ ]:
# Imprimimos un ejemplo de transacciones para cada cliente
for customer_id, transacciones in transaccion_cliente.items():
    print(f"Transacciones del cliente {customer_id}:")

    # Contador para rastrear el número de transacciones impresas
    contador_transacciones = 0

    for i, transaccion in enumerate(transacciones):
        # Imprimir solo las primeras 1000 transacciones debido al espacio
        if contador_transacciones < 1000:
            print(transaccion)
            contador_transacciones += 1
        else:
            break

    print("----")


Se han truncado las últimas 5000 líneas del flujo de salida.
['ALARM CLOCK BAKELIKE GREEN', Timestamp('2011-02-22 16:13:00')]
["FANNY'S REST STOPMETAL SIGN", Timestamp('2011-02-22 16:13:00')]
['HOME SWEET HOME METAL SIGN ', Timestamp('2011-02-22 16:13:00')]
["YOU'RE CONFUSING ME METAL SIGN ", Timestamp('2011-02-22 16:13:00')]
['CHOCOLATE THIS WAY METAL SIGN', Timestamp('2011-02-22 16:13:00')]
['SOLDIERS EGG CUP ', Timestamp('2011-02-22 16:13:00')]
['GLITTER BUTTERFLY CLIPS', Timestamp('2011-02-22 16:13:00')]
['6 CHOCOLATE LOVE HEART T-LIGHTS', Timestamp('2011-02-22 16:13:00')]
['RIDGED GLASS T-LIGHT HOLDER', Timestamp('2011-02-22 16:13:00')]
['HANGING JAM JAR T-LIGHT HOLDER', Timestamp('2011-02-22 16:13:00')]
['SCALLOP SHELL SOAP DISH', Timestamp('2011-02-22 16:13:00')]
['CLAM SHELL SMALL ', Timestamp('2011-02-22 16:13:00')]
['BLUE JUICY FRUIT PHOTO FRAME', Timestamp('2011-02-22 16:13:00')]
['GREEN JUICY FRUIT PHOTO FRAME', Timestamp('2011-02-22 16:13:00')]
['TWO DOOR CURIO CABINET', T


Debido al volumen considerable de datos, hemos optado por trabajar únicamente con un porcentaje de ellos. Esto se debe a que, de lo contrario, nuestra capacidad de memoria no sería suficiente para manejarlos todos.

In [7]:
datos_2000 = datos.head(2000)
# Ordenar las transacciones por CustomerID y luego por InvoiceDate
datos_reducido = datos_2000.sort_values(by=['CustomerID', 'InvoiceDate'])
# Creamos una lista de transacciones para cada cliente
transaccion_cliente = {}

# Iteramos sobre cada grupo de transacciones por cliente
for customer_id, group in datos_reducido.groupby('CustomerID'):

    # Obtenemos una lista de transacciones ordenadas por fecha para este cliente
    transacciones = group[['StockCode', 'Description', 'InvoiceDate']].values.tolist()

    # Agregamos la lista de transaciones como una secuencia para este cliente en el diccionario
    transaccion_cliente[customer_id] = transacciones

Pasamos las transacciones a un formato para utilizar por el algoritmo GSP (Generalized Sequential Patterns).

Cada sub-lista dentro de sequences representa la secuencia de códigos de producto comprados por un cliente específico en el orden en que fueron comprados. Este formato de lista  es necesario para el uso de algoritmo GSP de la librería gsppy. GSP coge una lista de secuencias y encuentra los patrones frecuentes.

In [8]:
# Convertimos las transacciones a un formato adecuado para GSP
sequences = []
for transacciones in transaccion_cliente.values():
    sequence = [transaccion[0] for transaccion in transacciones]
    sequences.append(sequence)

## **3. Aplicación de GSP**

Implementamos el algoritmo GSP para encontrar patrones secuenciales frecuentes.

El resultado es una lista de diccionarios donde cada diccionario representa un conjunto de patrones secuenciales de una longitud determinada, desde k=1 hasta k=4. Las claves son los patrones encontrados y los valores son el soporte correspondiente a cada patrón (la proporción de secuencias que contienen ese patrón).

In [9]:
# Configuramos y ejecutamos GSP
gsp = GSP(sequences)
resultado_gsp = gsp.search(minsup=0.1)

# Verificamos la estructura del resultado
print(resultado_gsp)

# Se está configurando y ejecutando el algoritmo GSP.

[{(22866,): 10, (22632,): 12, (22865,): 9, (22961,): 12, (21212,): 8, (22197,): 8, (22900,): 8, (22837,): 8, (22969,): 8, (22867,): 8, (22910,): 9, (22086,): 9, (84879,): 8, (22960,): 8, ('85123A',): 8, ('85099B',): 8, (22423,): 8}]


## **K=1**

Creamos dos configuraciones diferentes del algoritmo GSP y luego ejecutamos el algoritmo con cada configuración, imprimiendo los patrones encontrados junto con su soporte para cada configuración.

1.- Configuración 1: Soporte mínimo del 10%:

- Definimos soporte_min_1 como 0.1, lo que corresponde al 10%.
- Creamos un objeto gsp_1 utilizando la clase GSP y pasamos la lista de secuencias sequences.
- Ejecutamos el método search() de gsp_1 estableciendo el umbral de soporte mínimo.

2.- Configuración 2: Soporte mínimo del 20%:

- Definimos soporte_min_2 como 0.20, lo que corresponde al 20%.
- Creamos un nuevo objeto gsp_2 utilizando la clase GSP y se pasa la misma lista de secuencias sequences.
- Ejecutamos el método search() de gsp_2 estableciendo el umbral de soporte mínimo diferente.

In [10]:
# Configuración 1: Soporte mínimo del 10%
soporte_min_1 = 0.1
gsp_1 = GSP(sequences)
patron_1 = gsp_1.search(soporte_min_1)

# Configuración 2: Soporte mínimo del 15%
soporte_min_2 = 0.2
gsp_2 = GSP(sequences)
patron_2 = gsp_2.search(minsup=soporte_min_2)

# Imprimimos los resultados para la configuración 1
print(f"Resultados con soporte mínimo del 10%:")
for patron_dict in patron_1:
    for patron, soporte in patron_dict.items():
        print(f"Patrón: {patron}, Soporte: {soporte}")

# Imprimimos los resultados para la configuración 2
print(f"\nResultados con soporte mínimo del 20%:")
for patron_dict in patron_2:
    for patron, soporte in patron_dict.items():
        print(f"Patrón: {patron}, Soporte: {soporte}")

Resultados con soporte mínimo del 10%:
Patrón: (22632,), Soporte: 12
Patrón: (22866,), Soporte: 10
Patrón: (22865,), Soporte: 9
Patrón: (22961,), Soporte: 12
Patrón: (21212,), Soporte: 8
Patrón: (22197,), Soporte: 8
Patrón: (22900,), Soporte: 8
Patrón: (22837,), Soporte: 8
Patrón: (22969,), Soporte: 8
Patrón: (22867,), Soporte: 8
Patrón: (22910,), Soporte: 9
Patrón: (22086,), Soporte: 9
Patrón: (84879,), Soporte: 8
Patrón: (22960,), Soporte: 8
Patrón: ('85123A',), Soporte: 8
Patrón: ('85099B',), Soporte: 8
Patrón: (22423,), Soporte: 8

Resultados con soporte mínimo del 20%:


## **K=2**

Continuando con K=2 procedemos con un soporte mínimo y las transacciones como entrada. Luego iteramos cada transacción y elucubramos las combinaciones de  2. Filtramos aquellos que pasan el corte que anteriormente pasaron las combinaciones de K=1 y concluimos las combinaciones de K=2

In [11]:
from itertools import combinations

def find_frequent_sequences_k2(transacciones, minsup):

    # Contamos la frecuencia de cada par de elementos en las transacciones
    contador_parejas = {}
    for transaccion in transacciones:
        grupo_parejas = combinations(transaccion, 2)  # Generamos todas las combinaciones de longitud 2
        for pareja in grupo_parejas:
            if pareja in contador_parejas:
                contador_parejas[pareja] += 1
            else:
                contador_parejas[pareja] = 1

    # Filtramos los pares que cumplen con el umbral de soporte mínimo
    numero_transacciones = len(transacciones)
    frequent_grupo_parejas = {pareja: count / numero_transacciones for pareja, count in contador_parejas.items() if count / numero_transacciones >= minsup}

    return frequent_grupo_parejas

# Configuración 1: Soporte mínimo del 10%
soporte_min_1 = 0.1
frequent_grupo_parejas_1 = find_frequent_sequences_k2(sequences, soporte_min_1)

# Configuración 2: Soporte mínimo del 15%
soporte_min_2 = 0.15
frequent_grupo_parejas_2 = find_frequent_sequences_k2(sequences, soporte_min_2)

# Imprimimos los resultados para la configuración 1
print(f"Resultados con soporte mínimo del 10%:")
for pareja, soporte in frequent_grupo_parejas_1.items():
    print(f"Patrón: {pareja}, Soporte: {soporte}")

# Imprimimos los resultados para la configuración 2
print(f"\nResultados con soporte mínimo del 20%:")
for pareja, soporte in frequent_grupo_parejas_2.items():
    print(f"Patrón: {pareja}, Soporte: {soporte}")

Resultados con soporte mínimo del 10%:
Patrón: (22866, 21212), Soporte: 0.1232876712328767
Patrón: (22633, 22632), Soporte: 0.1780821917808219
Patrón: (22633, '85123A'), Soporte: 0.1780821917808219
Patrón: (22632, '85123A'), Soporte: 0.1917808219178082
Patrón: (22752, 22632), Soporte: 0.2054794520547945
Patrón: (22632, 22633), Soporte: 0.2191780821917808
Patrón: (22866, '85231B'), Soporte: 0.1095890410958904
Patrón: (22632, 37370), Soporte: 0.1780821917808219
Patrón: (22632, 22632), Soporte: 0.1506849315068493
Patrón: (37370, 22632), Soporte: 0.136986301369863
Patrón: (82486, 22632), Soporte: 0.136986301369863
Patrón: (21871, 22632), Soporte: 0.136986301369863
Patrón: (21871, '84029E'), Soporte: 0.1506849315068493
Patrón: (21871, '84029G'), Soporte: 0.1506849315068493
Patrón: (21871, 22633), Soporte: 0.136986301369863
Patrón: (21871, 21068), Soporte: 0.1506849315068493
Patrón: (21871, 21071), Soporte: 0.1506849315068493
Patrón: (22077, 22900), Soporte: 0.1095890410958904
Patrón: (22632

## **K=3**

Realizamos lo mismo que en el anterior paso, solo que iteramos sobre cada transacción buscando todas las combinaciones de longitud 3.

In [12]:
from itertools import combinations

def find_frequent_sequences_k3(transacciones, minsup):
    # Contamos la frecuencia de cada trio de elementos en las transacciones
    contador_trios = {}
    for transaccion in transacciones:
        grupos_trios = combinations(transaccion, 3)  # Generar todas las combinaciones de longitud 3
        for trios in grupos_trios:
            if trios in contador_trios:
                contador_trios[trios] += 1
            else:
                contador_trios[trios] = 1

    # Filtramos las grupos_trios que cumplen con el umbral de soporte mínimo
    numero_transacciones = len(transacciones)
    frequent_grupos_trios = {trios: count / numero_transacciones for trios, count in contador_trios.items() if count / numero_transacciones >= minsup}

    return frequent_grupos_trios

# Configuración 1: Soporte mínimo del 10%
soporte_min_1 = 0.1
frequent_grupos_trios_1 = find_frequent_sequences_k3(sequences, soporte_min_1)

# Configuración 2: Soporte mínimo del 15%
soporte_min_2 = 0.15
frequent_grupos_trios_2 = find_frequent_sequences_k3(sequences, soporte_min_2)

# Imprimimos los resultados para la configuración 1
print(f"Resultados con soporte mínimo del 10%:")
for trios, soporte in frequent_grupos_trios_1.items():
    print(f"Patrón: {trios}, Soporte: {soporte}")

# Imprimimos los resultados para la configuración 2
print(f"\nResultados con soporte mínimo del 20%:")
for trios, soporte in frequent_grupos_trios_2.items():
    print(f"Patrón: {trios}, Soporte: {soporte}")

Se han truncado las últimas 5000 líneas del flujo de salida.
Patrón: ('82494L', 22752, 21071), Soporte: 0.136986301369863
Patrón: ('82494L', 22752, 21068), Soporte: 0.136986301369863
Patrón: ('82494L', 22752, 82483), Soporte: 0.136986301369863
Patrón: ('82494L', 22752, 82486), Soporte: 0.136986301369863
Patrón: ('82494L', 22752, 82482), Soporte: 0.136986301369863
Patrón: ('82494L', 22752, '82494L'), Soporte: 0.136986301369863
Patrón: ('82494L', 22752, '84029G'), Soporte: 0.136986301369863
Patrón: ('82494L', 22752, '84029E'), Soporte: 0.136986301369863
Patrón: ('82494L', 22752, 22752), Soporte: 0.136986301369863
Patrón: ('82494L', 22752, 22632), Soporte: 0.2602739726027397
Patrón: ('82494L', 22752, 22633), Soporte: 0.2602739726027397
Patrón: ('82494L', 22752, 22803), Soporte: 0.2191780821917808
Patrón: ('82494L', 21730, '85123A'), Soporte: 0.136986301369863
Patrón: ('82494L', 21730, 71053), Soporte: 0.136986301369863
Patrón: ('82494L', 21730, '84406B'), Soporte: 0.136986301369863
Patrón

## **k=4**

Realizamos exactamente lo mismo pero buscando las combinaciones de 4 elementos.

In [ ]:
def find_frequent_sequences_k4(transacciones, minsup):
    # Contamos la frecuencia de cada cuarteto de elementos en las transacciones
    contador_cuartetos = {}
    for transaccion in transacciones:
        grupos_cuartetos = combinations(transaccion, 4)  # Generamos todas las combinaciones de longitud 4
        for cuartetos in grupos_cuartetos:
            if cuartetos in contador_cuartetos:
                contador_cuartetos[cuartetos] += 1
            else:
                contador_cuartetos[cuartetos] = 1

    # Filtramos las grupos_cuartetos que cumplen con el umbral de soporte mínimo
    numero_transacciones = len(transacciones)
    frequent_grupos_cuartetos = {cuartetos: count / numero_transacciones for cuartetos, count in contador_cuartetos.items() if count / numero_transacciones >= minsup}

    return frequent_grupos_cuartetos

# Configuración 1: Soporte mínimo del 10%
soporte_min_1 = 0.1
frequent_grupos_cuartetos_1 = find_frequent_sequences_k4(sequences, soporte_min_1)

# Configuración 2: Soporte mínimo del 15%
soporte_min_2 = 0.15
frequent_grupos_cuartetos_2 = find_frequent_sequences_k4(sequences, soporte_min_2)

# Imprimimos los resultados para la configuración 1
print(f"Resultados con soporte mínimo del 10%:")
for cuartetos, soporte in frequent_grupos_cuartetos_1.items():
    print(f"Patrón: {cuartetos}, Soporte: {soporte}")

# Imprimimos los resultados para la configuración 2
print(f"\nResultados con soporte mínimo del 20%:")
for cuartetos, soporte in frequent_grupos_cuartetos_2.items():
    print(f"Patrón: {cuartetos}, Soporte: {soporte}")

Cuando intentamos buscar con k = 4 no se pueden generar más secuencias. Al no encontrar se habría acabado el problema.

## **CONCLUSIONES**

Como conclusiones sacamos
- Database y transacciones: usando la database llamada Online Retail.xlsx, hemos realizado transacciones de los distintos clientes, en función de el momento de compra, para posteriormente analizar el modo de compra en distintos momentos.
- GSP: Hemos hecho uso del algoritmo GSP con dos soportes mínimos distintos para comparar como actuaba cada uno de ellos
- Patrones: Por último hemos interpretado algunos patrones secuenciales.



El patrón más común depende del valor mínimo de soporte y de la longitud de la secuencia considerada. Por ejemplo, si k = 1 y el soporte mínimo es del 10%, los patrones más comunes son números simples como "22866", "22632" y "22961". Sin embargo, a medida que aumenta la longitud de la secuencia (k = 2, k = 3), los patrones tienden a volverse más complejos e incluyen combinaciones de diferentes elementos.

Vemos que aumentar el valor mínimo de soporte reduce la cantidad de patrones comunes detectados. Por ejemplo, cuando comparamos resultados con el mismo valor de k pero diferentes valores mínimos de soporte, encontramos que el número de patrones identificados disminuye a medida que aumenta el umbral mínimo de soporte. Además, vemos que la complejidad computacional del algoritmo aumenta significativamente con la longitud de la secuencia (k). Por lo tanto, analizar los resultados del algoritmo GSP puede proporcionar información importante sobre los patrones de secuencia en los datos, lo que puede ayudar a comprender el comportamiento del usuario y tomar decisiones informadas en diversas aplicaciones comerciales.